# Use pretrained model for test predictions

In [2]:
!pip3 install kenlm
!pip3 install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://packagecloud.io/github/git-lfs/pypi/simple
DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://packagecloud.io/github/git-lfs/pypi/simple


DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [ ]:
import torch

from utils import SR
from datasets import Audio, load_dataset
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

# DL data
dataset_name = "lucas-meyer/asr_af"
af_test_set = load_dataset(dataset_name, split="test")
af_test_set = af_test_set.cast_column("audio", Audio(sampling_rate=SR))

# DL model
repo_name = "lucas-meyer/wav2vec2-xls-r-300m-asr_af"
model = Wav2Vec2ForCTC.from_pretrained(repo_name)
processor = Wav2Vec2Processor.from_pretrained(repo_name)

# DL model with LM
repo_name = "lucas-meyer/wav2vec2-xls-r-300m-with-LM-asr_af"
model_with_LM = Wav2Vec2ForCTC.from_pretrained(repo_name)
processor_with_LM = Wav2Vec2Processor.from_pretrained(repo_name)

2023-09-21 09:22:04.998548: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-21 09:22:06.399932: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
for i in range(10):
    input_values = processor(af_test_set[i]["audio"]["array"], sampling_rate=SR).input_values[0]
    input_dict = processor(input_values,
                           sampling_rate=SR,
                           return_tensors="pt",
                           padding=True)

    logits = model(input_dict.input_values).logits
    logits = logits.detach()
    pred_ids = torch.argmax(logits, dim=-1)[0]

    pred = processor.decode(pred_ids)
    true = af_test_set[i]["transcription"].lower()

    print(f"Test {i}:")
    print(f"  - pred: {pred}")
    print(f"  - true: {true}\n")

In [ ]:
for i in range(10):
    input_values = processor_with_LM(af_test_set[i]["audio"]["array"], sampling_rate=SR).input_values[0]
    input_dict = processor_with_LM(input_values,
                           sampling_rate=SR,
                           return_tensors="pt",
                           padding=True)

    logits = model_with_LM(input_dict.input_values).logits
    logits = logits.detach()
    pred_ids = torch.argmax(logits, dim=-1)[0]

    pred = processor_with_LM.decode(pred_ids)
    true = af_test_set[i]["transcription"].lower()

    print(f"Test {i}:")
    print(f"  - pred: {pred}")
    print(f"  - true: {true}\n")

In [ ]:
true_transcriptions = []
model_predictions = []
model_with_LM_predictions = []

for i in range(200):
    # Add true transcription
    true_transcriptions.append(af_test_set[i]["transcription"].lower())

    # Add model prediction
    input_values = processor(af_test_set[i]["audio"]["array"], sampling_rate=SR).input_values[0]
    input_dict = processor(input_values,
                           sampling_rate=SR,
                           return_tensors="pt",
                           padding=True)
    logits = model(input_dict.input_values).logits
    logits = logits.detach()
    pred_ids = torch.argmax(logits, dim=-1)[0]
    pred = processor.decode(pred_ids)

    # Add model with LM prediction
    input_values = processor_with_LM(af_test_set[i]["audio"]["array"], sampling_rate=SR).input_values[0]
    input_dict = processor_with_LM(input_values,
                           sampling_rate=SR,
                           return_tensors="pt",
                           padding=True)
    logits = model_with_LM(input_dict.input_values).logits
    logits = logits.detach()
    pred_ids = torch.argmax(logits, dim=-1)[0]
    pred = processor_with_LM.decode(pred_ids)

In [ ]:
from evaluate import load

wer = load("wer")
wer_score_model = wer.compute(predictions=model_predictions, references=true_transcriptions)
wer_score_model_with_LM = wer.compute(predictions=model_with_LM_predictions, references=true_transcriptions)

print(wer_score_model)
print(wer_score_model_with_LM)